# Mortality by place, Italy

## Introduction

In this analysis using **pdnd-nteract** we will present some data regarding the current state (updated to 2016) and evolution in recent years of causes of death in Italy, both from a national and from provincial level.

This dataset is provided by the Italian National Institute of Statistics (ISTAT), which is the main producer of official statistics in Italy and it contains Vital statistics on causes of death: The Vital statistics system on causes of death is the main source for the evaluation of the health status of the population, and for the health programs and resources allocation.

Given the considerable size of the [complete dataset](http://dati.istat.it/Index.aspx?DataSetCode=DCIS_CMORTE1_EV), we will use multiple subsets downloaded from the ISTAT tool.

## Let's start
We will start by importing some python libraries:


In [1]:
import requests
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from io import StringIO
pd.options.display.html.table_schema = True
pd.set_option('display.max_rows', None)
import os

In [2]:
national_df = pd.read_csv(os.path.join('mortality', 'italia2016.csv'))
national_df.head(20)

,ITTER107,Territorio,TIPO_DATO15,Tipo dato,SEXISTAT1,Sesso,CAUSEMORTE_SL,Causa iniziale di morte - European Short List,TIME,Seleziona periodo,Value,Flag Codes,Flags
0,IT,Italia,DEATH,morti,1,maschi,2_1_8,"di cui tumori maligni della trachea, dei bronc...",2016,2016,24059,NaN,NaN
1,IT,Italia,DEATH,morti,1,maschi,2_1_14,di cui tumori maligni della prostata,2016,2016,7540,NaN,NaN
2,IT,Italia,DEATH,morti,1,maschi,7,malattie del sistema circolatorio,2016,2016,96728,NaN,NaN
3,IT,Italia,DEATH,morti,1,maschi,7_1_2,du cui altre malattie ischemiche del cuore,2016,2016,21851,NaN,NaN
4,IT,Italia,DEATH,morti,1,maschi,7_4,altre malattie del sistema circolatorio,2016,2016,17799,NaN,NaN
5,IT,Italia,DEATH,morti,1,maschi,8_2,polmonite,2016,2016,5286,NaN,NaN
6,IT,Italia,DEATH,morti,1,maschi,17_1_2,di cui cadute accidentali,2016,2016,2060,NaN,NaN
7,IT,Italia,DEATH,morti,1,maschi,17_1_5,di cui altri accidenti,2016,2016,4945,NaN,NaN
8,IT,Italia,DEATH,morti,1,maschi,2_1_4,"di cui tumori maligni del colon, del retto e d...",2016,2016,10652,NaN,NaN
9,IT,Italia,DEATH,morti,1,maschi,2_1_7,di cui tumori maligni della laringe,2016,2016,1424,NaN,NaN


In [3]:
len(national_df)

32214

In [4]:
national_df['Sesso'].unique()

array(['maschi', 'totale', 'femmine'], dtype=object)

In [5]:
len(national_df['Causa iniziale di morte - European Short List'].unique())

87

In [6]:
list(national_df)

['ITTER107',
 'Territorio',
 'TIPO_DATO15',
 'Tipo dato',
 'SEXISTAT1',
 'Sesso',
 'CAUSEMORTE_SL',
 'Causa iniziale di morte - European Short List',
 'TIME',
 'Seleziona periodo',
 'Value',
 'Flag Codes',
 'Flags']

In [12]:
national_df.columns = ['causa' if x=='Causa iniziale di morte - European Short List' else 
                       'causa_cod' if x=='CAUSEMORTE_SL' else 
                       x for x in national_df.columns]

Here we can see a *bar chart* showing the top macro-causes of death.

In [19]:
national_total_df = national_df[national_df['Sesso'].isin(['totale']) & national_df['Territorio'].isin(['Italia'])]
national_total_df = national_total_df[~national_total_df['causa'].isin(['totale'])]
national_total_df_macro = national_total_df[~national_total_df['causa_cod'].str.contains('_')]
national_total_df_macro = national_total_df_macro[['causa','Value']]
national_total_df_macro

,causa,Value
172,malattie del sistema circolatorio,221914
174,alcune condizioni morbose che hanno origine ne...,891
176,cause esterne di traumatismo e avvelenamento,24352
177,alcune malattie infettive e parassitarie,12850
185,malattie della cute e del tessuto sottocutaneo,1214
188,malformazioni congenite ed anomalie cromosomiche,1232
190,tumori,179502
192,disturbi psichici e comportamentali,21535
193,malattie dell'apparato digerente,22643
201,malattie del sistema respiratorio,46537


This chart shows the top 15 causes broken down in more detail. They are filtered using a regular expression on the `CAUSEMORTE_SL` column.

In [33]:
national_total_df_detailed = national_total_df[national_total_df['causa_cod'].str.match('^\d+_\d+$')]
national_total_df_detailed = national_total_df_detailed[['causa','Value']]
national_total_df_detailed.sort_values('Value', ascending = False).head(15)

,causa,Value
179,tumori maligni,170286
255,malattie ischemiche del cuore,66868
236,malattie cerebrovascolari,56958
200,altre malattie del cuore,51275
216,altre malattie del sistema circolatorio,46813
208,malattie croniche delle basse vie respiratorie,23710
182,diabete mellito,21354
183,demenza,19875
203,accidenti,19508
194,altre malattie dell'apparato digerente,16191


With tumors being among the most common cause of death by a considerable margin, we can plot a chart of the incidence by type and sex.

In [44]:
national_mf = national_df[~national_df['Sesso'].isin(['totale']) & national_df['Territorio'].isin(['Italia'])]
national_mf = national_mf[~national_mf['causa'].isin(['totale'])]
national_mf_tumors = national_mf[national_mf['causa'].str.contains('di cui tumori')]
national_mf_tumors = national_mf_tumors[['Sesso','causa','Value']]
national_mf_tumors

,Sesso,causa,Value
0,maschi,"di cui tumori maligni della trachea, dei bronc...",24059
1,maschi,di cui tumori maligni della prostata,7540
8,maschi,"di cui tumori maligni del colon, del retto e d...",10652
9,maschi,di cui tumori maligni della laringe,1424
10,maschi,di cui tumori maligni del seno,144
16,maschi,di cui tumori maligni della vescica,4883
26,maschi,di cui tumori maligni dello stomaco,5458
27,maschi,di cui tumori maligni del pancreas,5834
38,maschi,"di cui tumori maligni delle labbra, cavità ora...",2228
46,maschi,di cui tumori maligni dell'esofago,1417


In [45]:
national_mf_tumors_m = national_mf_tumors.merge(right=national_mf_tumors, left_on='causa', right_on='causa')
#df2df2 = df.merge(right=df, left_on='buyer_name', right_on='seller_name')
national_mf_tumors_m

,Sesso_x,causa,Value_x,Sesso_y,Value_y
0,maschi,"di cui tumori maligni della trachea, dei bronc...",24059,maschi,24059
1,maschi,"di cui tumori maligni della trachea, dei bronc...",24059,femmine,9779
2,femmine,"di cui tumori maligni della trachea, dei bronc...",9779,maschi,24059
3,femmine,"di cui tumori maligni della trachea, dei bronc...",9779,femmine,9779
4,maschi,di cui tumori maligni della prostata,7540,maschi,7540
5,maschi,"di cui tumori maligni del colon, del retto e d...",10652,maschi,10652
6,maschi,"di cui tumori maligni del colon, del retto e d...",10652,femmine,8923
7,femmine,"di cui tumori maligni del colon, del retto e d...",8923,maschi,10652
8,femmine,"di cui tumori maligni del colon, del retto e d...",8923,femmine,8923
9,maschi,di cui tumori maligni della laringe,1424,maschi,1424
